# 라이브러리 호출

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from gensim.models import Word2Vec, FastText

# 토큰화 함수 생성

In [2]:
def tokenization():

    data = pd.read_table("ratings_train.txt")[:10]
    
    tokenizer = Okt()

    def process_tokens(doc):
        results = tokenizer.pos(doc)

        tokens = []
        for r in results:
            token, pos = r
            if pos == 'Noun' or pos == 'Adjective':
                tokens.append(token)
        return tokens    

    data['tokens'] = data['document'].apply(lambda x: process_tokens(x))
    return data

# 코퍼스 불러오기 및 토큰화

In [3]:
data = tokenization()
data

,id,document,label,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[더빙, 진짜, 짜증나네요, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 가볍지]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[무재, 밓었, 다그, 래서, 추천]"
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 구먼, 솔직히, 재미, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[몬페, 의, 익살스런, 연기, 영화, 스파이더맨, 커스틴, 던스트, 이뻐]"
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0,"[막, 걸음, 마, 세, 초등학교, 학년, 생인, 영화, 반개, 아까, 움]"
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0,"[원작, 긴장감, 제대로]"
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0,"[별, 반개, 아깝다, 욕, 이응경, 길용우, 생활, 년, 정말, 발, 해도, 그것..."
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,"[액션, 없는데도, 재미, 있는, 몇, 안되는, 영화]"
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1,"[왜케, 평점, 낮은건데, 꽤, 볼, 데, 헐리우드, 화려함에만, 있나]"


In [4]:
tokenized_data = data['tokens']

# train word2vec model
wvmodel = Word2Vec(
    sentences = tokenized_data,
    vector_size = 5,
    min_count = 1,
    window = 5,
    sg = 1
)

###### INPORTANT #######
embeddings_wv = []
for tokens in tokenized_data:
    # print(tokens) 
    tokens_vectors = []
    for token in tokens:
        vectors_wv= wvmodel.wv[token]
        tokens_vectors.append(vectors_wv)
    
    # print(tokens_vectors)
    vectors_array = np.array(tokens_vectors)
    # print(vectors_array, vectors_array.shape)
    
    embedding = np.mean(vectors_array, axis=0)
    # print(embedding, embedding.shape)
    
    embeddings_wv.append(embedding)
    
embeddings_wv

[array([ 0.04979223, -0.03824242,  0.08305701, -0.07365821,  0.07947041],
       dtype=float32),
 array([0.05065962, 0.04645307, 0.01441879, 0.03698254, 0.00833625],
       dtype=float32),
 array([-0.11220447, -0.04343849,  0.01599637,  0.02022813, -0.05657949],
       dtype=float32),
 array([-0.07300535, -0.0180869 ,  0.05578434, -0.03114302,  0.01648486],
       dtype=float32),
 array([-0.03293857,  0.09085197, -0.00556926, -0.03217097,  0.00063491],
       dtype=float32),
 array([ 0.05247071,  0.04063801,  0.00414832, -0.0204814 ,  0.04937178],
       dtype=float32),
 array([ 0.0322179 ,  0.00092756,  0.04165773,  0.06277312, -0.03603516],
       dtype=float32),
 array([-0.00701245,  0.03758062,  0.06276277, -0.04765067,  0.03034518],
       dtype=float32),
 array([ 0.00536731, -0.01174485, -0.02453074, -0.01656558, -0.09799786],
       dtype=float32),
 array([ 0.0384137 ,  0.04792345,  0.03034942, -0.02701363, -0.04635359],
       dtype=float32)]

In [6]:

embeddings_array = np.array(embeddings_wv)
embeddings_array

array([[ 0.04979223, -0.03824242,  0.08305701, -0.07365821,  0.07947041],
       [ 0.05065962,  0.04645307,  0.01441879,  0.03698254,  0.00833625],
       [-0.11220447, -0.04343849,  0.01599637,  0.02022813, -0.05657949],
       [-0.07300535, -0.0180869 ,  0.05578434, -0.03114302,  0.01648486],
       [-0.03293857,  0.09085197, -0.00556926, -0.03217097,  0.00063491],
       [ 0.05247071,  0.04063801,  0.00414832, -0.0204814 ,  0.04937178],
       [ 0.0322179 ,  0.00092756,  0.04165773,  0.06277312, -0.03603516],
       [-0.00701245,  0.03758062,  0.06276277, -0.04765067,  0.03034518],
       [ 0.00536731, -0.01174485, -0.02453074, -0.01656558, -0.09799786],
       [ 0.0384137 ,  0.04792345,  0.03034942, -0.02701363, -0.04635359]],
      dtype=float32)

In [7]:
data['embedding'] = embeddings_wv
data = data.drop('tokens', axis=1)
data

,id,document,label,embedding
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[0.049792226, -0.03824242, 0.08305701, -0.0736..."
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[0.050659623, 0.046453074, 0.014418787, 0.0369..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[-0.11220447, -0.04343849, 0.015996374, 0.0202..."
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[-0.07300535, -0.018086895, 0.05578434, -0.031..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[-0.03293857, 0.09085197, -0.0055692643, -0.03..."
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0,"[0.05247071, 0.040638015, 0.0041483236, -0.020..."
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0,"[0.0322179, 0.0009275551, 0.041657727, 0.06277..."
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0,"[-0.0070124487, 0.037580624, 0.06276277, -0.04..."
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,"[0.005367311, -0.01174485, -0.024530739, -0.01..."
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1,"[0.0384137, 0.047923446, 0.030349424, -0.02701..."
